# Testing Run Length Encoding


## Imports


In [159]:
from importlib.util import spec_from_loader, module_from_spec
from importlib.machinery import SourceFileLoader
from scipy.io import wavfile
from glob import glob
import numpy as np
import sys

# Import Encode
spec = spec_from_loader("encode", SourceFileLoader("encode", "../../encode"))
encode = module_from_spec(spec)
spec.loader.exec_module(encode)

# Import Decode
spec = spec_from_loader("decode", SourceFileLoader("decode", "../../decode"))
decode = module_from_spec(spec)
spec.loader.exec_module(decode)

## Function Definitions


In [160]:
def encode_via_rle(rle_l_raw: list):
    """This algorithm will search for contiguous values within the
       array. When the rle_location_count is greater than the value
       65530, the count is reduced by this value in order to
       prevent an overflow of an unsigned 16-bit integer. This allows
       for the data to be stored with 2 bytes when the format of the
       array is a known value in advance of decoding this format. The
       choice of integer 65530 is an arbitrary value less than that of
       the maximum value of an unsigned 16-bit integer (65536).

    Args:
        rle_l_raw (list): This is a list of integer values to be
                          encoded.

    Returns:

        index_array (list): This is the list of run length encoded
                            values.
        rle_locations (list): This is a list of locations of elements
                              that are repeated that are present in the
                              array of indices.
    """
    initial_index = 0
    second_index = 1
    count = 0
    index_array = []
    rle_locations = []
    rle_location_count = 0

    while second_index < len(rle_l_raw):
        if rle_l_raw[initial_index] == rle_l_raw[second_index]:
            index_array.append(rle_l_raw[initial_index])
            rle_locations.append(rle_location_count)

            # continue searching the breadth of the array; increasing
            # count
            while (
                second_index <= len(rle_l_raw)
                and rle_l_raw[initial_index] == rle_l_raw[second_index]
            ):
                count += 1
                second_index += 1
            index_array.append(count)
            if rle_location_count > 65530:
                rle_location_count -= 65530
            rle_location_count += 2
        else:
            index_array.append(rle_l_raw[initial_index])
            if rle_location_count > 65530:
                rle_location_count -= 65530
            rle_location_count += 1
        count = 0
        initial_index = second_index
        second_index += 1
    if rle_l_raw[-1] != index_array[-2]:
        index_array.append(rle_l_raw[-1])
    return index_array, rle_locations

In [161]:
def identify_index_split(rle_locations, verbose=False):
    """This will identify where the rle_location values reset to a value
       that is below 65530.
    Args:
        rle_locations (list): This is a list of locations of values that
                              are encoded in the index_array.

    Returns:
        split_indices (list): This is a list of indices where the values
                              are reset in the rle_locations_array.
    """
    split_indices = []
    for index, value in enumerate(rle_locations):
        if index == 0:
            prev_value = value
            continue
        if value > prev_value:
            prev_value = value
        else:
            if verbose:
                print(f"index: {index}")
                print(f"prev_value: {prev_value}")
                print(f"value: {value}")
            split_indices.append(index)
            prev_value = value
    return split_indices

In [162]:
# Expand RLE Algorithm
def decode_rle(rle_locations: list, index_array: list):
    """This will expand the index array where the values have been
       compressed by run-length-encoding.

    Args:
        rle_locations (list): This is a list of locations in the
                              index_array where the locations have been
                              run-length-encoded. The subsequent values
                              of these locations are frequencies of the
                              run-length-encoded values.
        index_array (list): This is a list of values which contain
                            either signular values or run-length-encoded
                            values followed by the frequency of the
                            run-length-encoded value.
    """
    reconstructed_array = []
    rle_index = 0
    continue_current_index_past_frequency_value = False

    for current_index in range(0, len(index_array)):
        if continue_current_index_past_frequency_value:
            continue_current_index_past_frequency_value = False
            continue
        if rle_index <= (len(rle_locations) - 1):
            if current_index > 131060:  # twice the cutoff value of 65530
                rle_location_index = rle_locations[rle_index] + 131060
            elif current_index > 65530 and current_index <= 131060:
                rle_location_index = rle_locations[rle_index] + 65530
            else:  # current_index is below 65530
                rle_location_index = rle_locations[rle_index]
        if current_index != rle_location_index:
            reconstructed_array.append(index_array[current_index])
        else:
            if rle_index <= (len(rle_locations) - 1):
                run_length_expanded_l = [
                    index_array[rle_location_index]
                    for x in range((index_array[rle_location_index + 1]) + 1)
                ]
                reconstructed_array.extend(run_length_expanded_l)
                rle_index += 1
                continue_current_index_past_frequency_value = True
    return reconstructed_array

In [163]:
def format_encoded_rle_to_bytes(index_array: list, rle_locations: list):
    """This is the encoding driver of the run_length_encoded array.

    Args:
        index_array (list): This is the compressed list of values which
                            include run-length-encoded values followed
                            by their frequency.
        rle_locations (list): This is a list of index locations that are
                              run-length-encoded in the index array.
                              The frequency of the run-length-encoded
                              value is defined as the
                              index_array[rle_locations[current_rle_index] + 1]
                              or the subsequent value from the
                              run-length-encoded value in the
                              index_array.

    Returns:
        format_encoded_rle_bytes (bytes): This is the string of bytes of
                                          the formatted
                                          run-length-encoded array.
    """
    format_encoded_rle = [len(rle_locations)]
    format_encoded_rle.extend(rle_locations)
    format_encoded_rle.extend(index_array)
    format_encoded_rle = np.array(format_encoded_rle, dtype=np.uint16)
    format_encoded_rle_bytes = format_encoded_rle.tobytes()
    return format_encoded_rle_bytes

In [164]:
def parse_formatted_rle_bytes(format_encoded_rle_bytes: bytes):
    """This is the encoding driver of the run_length_encoded array.

    Args:
        format_encoded_rle_bytes (bytes): This is the compressed list of
                                          values in the form of a byte
                                          array. The order of the values
                                          is as following: length of the
                                          run-length-encoded locations
                                          array, the run-length-encoded
                                          locations array, and the
                                          index_array.

    Returns:
        rle_locations (list): This is a list of unsigned 16-bit
                              integers.
        index_array (list): This is a list of unsigned 16-bit integers.
                            These values contain run-length-encoded
                            sequences where the run-length-encoded-value
                            is followed by the frequency of the value.
    """
    encoded_rle = np.frombuffer(format_encoded_rle_bytes, dtype=np.uint16)
    len_rle_locations = encoded_rle[0]

    # +1 to skip over the rle_locations length in the line below:
    rle_locations = encoded_rle[1 : len_rle_locations + 1]

    index_array = encoded_rle[(len_rle_locations + 1) :]
    return rle_locations, index_array

## Import Data & Huffman Encode Values


In [322]:
data_dir = "../../data/"
data_file_l = glob(data_dir + "*.wav")
current_file = data_file_l[0]

rate, data = wavfile.read(current_file)

data_bytes = data.tobytes()

compressed_file_path = "../../data/102b47d9-371e-412a-8995-0dc6115ab2bb.wav.brainwire"

encode.huffman_encoding(
    input_data=data_bytes,
    compressed_file_path="../../data/102b47d9-371e-412a-8995-0dc6115ab2bb.wav.brainwire",
)

with open(compressed_file_path, "rb") as fp:
    data_huffman_encoded_bytes = fp.read()
    fp.close()

# type(data_huffman_encoded_bytes)

## Testing Compressing a substring of the larger dataset


In [324]:
len(data_huffman_encoded_bytes_sub_string)

12

In [325]:
sys.getsizeof(data_huffman_encoded_bytes_sub_string)

45

In [327]:
data_huffman_encoded_bytes_sub_string

b'\x80\x04\x95\x87\x05\x00\x00\x00\x00\x00\x00}'

In [269]:
data_huffman_encoded_bytes_sub_string = data_huffman_encoded_bytes[:12]

rle_l_raw = []
for index, data in enumerate(data_huffman_encoded_bytes_sub_string):
    rle_l_raw.append(data)
rle_l_raw

[128, 4, 149, 135, 5, 0, 0, 0, 0, 0, 0, 125]

In [329]:
len(rle_l_raw)

12

In [330]:
sys.getsizeof(rle_l_raw)

184

In [271]:
sys.getsizeof(rle_l_raw[0])

28

In [274]:
type(rle_l_raw[0])

int

In [275]:
index_array, rle_locations = encode_via_rle(rle_l_raw)
reconstructed_array = decode_rle(rle_locations, index_array)

In [276]:
sys.getsizeof(reconstructed_array)

152

In [277]:
sys.getsizeof(reconstructed_array[0])

28

In [278]:
type(reconstructed_array[0])

int

In [280]:
reconstructed_array[0]

128

In [334]:
np.array(rle_l_raw, dtype=np.int16)

array([128,   4, 149, 135,   5,   0,   0,   0,   0,   0,   0, 125],
      dtype=int16)

In [281]:
# Test for equality in reconstructed array
not_equal = False
for index in range(0, len(rle_l_raw)):
    if rle_l_raw[index] != reconstructed_array[index]:
        not_equal = True
        print(f"Reconstructed Array is unequal at position: {index}")

if not_equal:
    print(f"Arrays are not equivalent.")
else:
    print(f"Arrays are equivalent.")

Arrays are equivalent.


In [282]:
formatted_encoded_rle_bytes = format_encoded_rle_to_bytes(index_array, rle_locations)

In [283]:
encoded_rle = np.frombuffer(formatted_encoded_rle_bytes, dtype=np.uint16)

In [285]:
rle_locations, index_array = parse_formatted_rle_bytes(formatted_encoded_rle_bytes)

In [286]:
reconstructed_array = decode_rle(rle_locations, index_array)

In [321]:
# Test for equality in reconstructed array
for index in range(0, len(rle_l_raw)):
    if rle_l_raw[index] != reconstructed_array[index]:
        print(f"Reconstructed Array is unequal at position: {index}")

In [288]:
len(formatted_encoded_rle_bytes)

20

In [289]:
len(data_huffman_encoded_bytes[:12])

12

In [ ]:
formatted_encoded_rle_bytes

b'\x01\x00\x05\x00\x80\x00\x04\x00\x95\x00\x87\x00\x05\x00\x00\x00\x05\x00}\x00'

In [311]:
import pandas as pd

In [312]:
data_array = []
for index, data in enumerate(formatted_encoded_rle_bytes):
    data_array.append(data)
    print(f"index: {index}\ndata: {data}\n")
formatted_encoded_rle_bytes_pd = pd.DataFrame(data_array)

index: 0
data: 1

index: 1
data: 0

index: 2
data: 5

index: 3
data: 0

index: 4
data: 128

index: 5
data: 0

index: 6
data: 4

index: 7
data: 0

index: 8
data: 149

index: 9
data: 0

index: 10
data: 135

index: 11
data: 0

index: 12
data: 5

index: 13
data: 0

index: 14
data: 0

index: 15
data: 0

index: 16
data: 5

index: 17
data: 0

index: 18
data: 125

index: 19
data: 0



In [316]:
formatted_encoded_rle_bytes_pd.columns = ["value"]

In [317]:
formatted_encoded_rle_bytes_pd.index.name = "row"

In [319]:
formatted_encoded_rle_bytes_pd

,value
row,
0,1
1,0
2,5
3,0
4,128
5,0
6,4
7,0
8,149


In [320]:
rle_l_raw

[128, 4, 149, 135, 5, 0, 0, 0, 0, 0, 0, 125]

In [291]:
data_huffman_encoded_bytes[:12]

b'\x80\x04\x95\x87\x05\x00\x00\x00\x00\x00\x00}'

In [292]:
data_array = []
for data in data_huffman_encoded_bytes:
    data_array.append(data)

In [293]:
sys.getsizeof(data_array[0])

28

In [294]:
max(data_array)

255

In [296]:
data_array = np.array(data_array, dtype=np.int16)

In [297]:
data_array

array([128,   4, 149, ..., 101,  46,  28], dtype=int16)

In [298]:
data_array[0]

128

In [299]:
print(f"Ratio of the formatted_encoded_rle_bytes ", end="")
print(f"to data_huffman_encoded_bytes: ", end="")
print(f"{len(formatted_encoded_rle_bytes) / len(data_huffman_encoded_bytes[:12])}")

Ratio of the formatted_encoded_rle_bytes to data_huffman_encoded_bytes: 1.6666666666666667


#### Principal Components Example of the Run-Length-Encoded Problem


In [194]:
# Original file size: 197 KB
# Huffman Encoded Compressed File Size: 130 KB
# RLE Encoded Version of the Huffman Encoded Compressed File is of Size: 272 KB

In [300]:
data_dir = "../../data/"
data_file_l = glob(data_dir + "*.wav")
current_file = data_file_l[0]

In [301]:
rate, data = wavfile.read(current_file)

data_bytes = data.tobytes()

compressed_file_path = "../../data/102b47d9-371e-412a-8995-0dc6115ab2bb.wav.brainwire"

encode.huffman_encoding(
    input_data=data_bytes,
    compressed_file_path="../../data/102b47d9-371e-412a-8995-0dc6115ab2bb.wav.brainwire",
)

with open(compressed_file_path, "rb") as fp:
    data_huffman_encoded_bytes = fp.read()
    fp.close()

In [302]:
sys.getsizeof(data)

197510

In [303]:
sys.getsizeof(data_bytes)

197431